# Testing SDTW

In [7]:
import numpy as np
import os 
import torch

from lib.nn_losses.soft_dtw_cuda import SoftDTW 

In [ ]:
# Initalize random X and y

# Create the sequences
batch_size, len_x, len_y, dims = 8, 15, 12, 5
x = torch.rand((batch_size, len_x, dims), requires_grad=True)
y = torch.rand((batch_size, len_y, dims))

# Create the "criterion" object
sdtw = SoftDTW(use_cuda=False, gamma=0.1)

# Compute the loss value
loss = sdtw(x, y)  # Just like any torch.nn.xyzLoss()
print("Loss value:", loss)
print("Loss mean:", loss.mean())

# Aggregate and call backward()
loss.mean().backward()

Loss value: tensor([ 7.1838,  7.8403,  7.7179,  7.1863,  9.6889,  8.8728, 10.1612,  8.1580],
       grad_fn=<_SoftDTWBackward>)
